### Basic RAG retriever using Langchain and Anthropic model

../notebooks/data/cleaned20242108.csv

In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
db = SQLDatabase.from_uri("sqlite:////Users/main/Desktop/database/llm.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM restaurants LIMIT 10;")

sqlite
['restaurants', 'restaurants_madrid']


"[(0, 'OPERATIONAL', 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', '#909CE1', 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'VP Jardín de Recoletos', 'ChIJ0U9LFZooQg0RgdlcHxZdkek', 4.5, 'ChIJ0U9LFZooQg0RgdlcHxZdkek', 'GOOGLE', 611.0, 'Calle de Gil de Santivañes, 6, Madrid', 40.4223075, -3.689168100000001, 40.42368518029149, -3.687793119708498, 40.42098721970849, -3.690491080291502, 1, 'C8C6+W8 Madrid, Spain', '8CGRC8C6+W8', None, None), (1, 'OPERATIONAL', 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', '#FF9E67', 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'New York Burger', 'ChIJ2fFEjZooQg0RbySrRH-t_nk', 4.5, 'ChIJ2fFEjZooQg0RbySrRH-t_nk', 'GOOGLE', 7357.0, 'Calle de Recoletos, 4, Madrid', 40.422131, -3.6908463, 40.4234758302915, -3.689437969708498, 40.4207778697085, -3.692135930291502, 1, 'C8C5+VM Madrid, Spain', '8CGRC8C5+VM', 2.0, None), (2, 'CLOSED_TEMPORARILY', 'h

In [3]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "how many restaurants are in the restaurants_madrid table? dont use double notation mark in the query for getting total_restaurants, toherwise you will not give me a result"})

'Error: (sqlite3.OperationalError) near "I": syntax error\n[SQL: I apologize for the confusion in my previous response. You\'re right, we don\'t need to use double quotes for the alias in this case. Here\'s the correct query:\n\nSQLQuery: SELECT COUNT(*) AS total_restaurants FROM restaurants_madrid]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [4]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many restaurants are in the restaurants_madrid table?"})

'I apologize, but I cannot provide an accurate answer to the question based on the information given. The SQL query execution resulted in an error, specifically a syntax error. This error occurred because the entire explanation was included in the SQL execution, rather than just the actual SQL query.\n\nThe correct SQL query to count the number of restaurants in the restaurants_madrid table would be:\n\n```sql\nSELECT COUNT(*) AS total_restaurants FROM restaurants_madrid\n```\n\nHowever, since this query was not successfully executed due to the error, we don\'t have the actual result to answer the question "How many restaurants are in the restaurants_madrid table?".\n\nTo get the correct answer, the SQL query needs to be executed properly without the additional text, and then we would need to see the result of that query.'